In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
weather = pd.read_csv("../Dataset/weather.csv",index_col="date",parse_dates=True)

In [3]:
weather.head()

,station_nbr,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
date,,,,,,,,,,,,,,,,,,,
2012-01-01,1,52,31,42,M,36,40,23,0,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
2012-01-01,2,48,33,41,16,37,39,24,0,0716,1626,RA,0.0,0.07,28.82,29.91,9.1,23,11.3
2012-01-01,3,55,34,45,9,24,36,20,0,0735,1720,,0.0,0.00,29.77,30.47,9.9,31,10.0
2012-01-01,4,63,47,55,4,28,43,10,0,0728,1742,,0.0,0.00,29.79,30.48,8.0,35,8.2
2012-01-01,6,63,34,49,0,31,43,16,0,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8


In [4]:
weather.shape

(20517, 19)

In [5]:
weather.replace(["M","-"],np.nan,inplace=True)
weather.snowfall.replace("  T",0.00,inplace=True)
weather.preciptotal.replace("  T",0.00,inplace=True)
weather.replace(r'^\s+$',np.nan,inplace=True,regex=True) # replace blank data with NaN

In [6]:
weather[(weather.tmin.isnull())&(weather.station_nbr==5)].shape

(822, 19)

In [7]:
weather[(weather.station_nbr==5)].shape

(852, 19)

In [8]:
weather[(weather.tmin.isnull())&(weather.station_nbr==5)].shape[0]/weather[(weather.station_nbr==5)].shape[0]

0.9647887323943662

In [9]:
#dropping all station #5 NaN readings.
weather = weather[(weather.station_nbr!=5) | ((weather.tmin.isnull() == False) & (weather.station_nbr==5))]

In [10]:
weather.loc[(weather.tmin.isnull() == True)&(weather.station_nbr==5)].shape

(0, 19)

In [11]:
weather.shape

(19695, 19)

In [12]:
20517-822

19695

In [13]:
#checkng the codesum still has codes that contains "M" in it.
df = weather.loc[weather.codesum.isnull() == False]
df [df.codesum.str.contains("M") == True].shape

(52, 19)

In [15]:
#convert all numbers from 'str' to 'float'
weather = weather.apply(pd.to_numeric,errors='ignore')

In [16]:
weather.isnull().sum()

station_nbr        0
tmax              84
tmin              86
tavg             647
depart         10689
dewpoint         224
wetbulb          430
heat             647
cool             647
sunrise         9625
sunset          9625
codesum        10229
snowfall        6402
preciptotal       38
stnpressure      107
sealevel        1281
resultspeed      147
resultdir        147
avgspeed          53
dtype: int64

In [18]:
org_null = weather.isnull().sum().sum()
org_null

51108

In [19]:
# Evaluate if tavg can be used to fill missing temps.
weather[(weather.tmax.isnull()) & (weather.tavg.isnull() == False)]

,station_nbr,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
date,,,,,,,,,,,,,,,,,,,


In [20]:
temp = weather[(weather.tmin.isnull()) & (weather.tavg.isnull() == False)]
temp

,station_nbr,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
date,,,,,,,,,,,,,,,,,,,
2012-10-23,9,63.0,NaN,32.0,NaN,NaN,NaN,33.0,0.0,NaN,NaN,RA BR HZ,NaN,0.00,29.00,29.77,4.8,12.0,5.6
2013-06-04,8,94.0,NaN,47.0,NaN,NaN,NaN,18.0,0.0,NaN,NaN,FG+ BR HZ,NaN,0.00,29.51,NaN,4.3,16.0,4.8
2013-11-24,15,26.0,NaN,13.0,-21.0,NaN,NaN,52.0,0.0,650.0,1615.0,NaN,0.1,0.02,29.46,NaN,NaN,NaN,19.2
2013-11-25,15,30.0,NaN,15.0,-19.0,NaN,NaN,50.0,0.0,651.0,1615.0,NaN,0.0,0.00,29.67,NaN,NaN,NaN,12.4
2013-12-06,20,33.0,NaN,17.0,NaN,NaN,NaN,48.0,0.0,NaN,NaN,FZRA BR,NaN,0.68,NaN,NaN,NaN,NaN,14.8


In [21]:
for i in range(len(temp)):
    print("Avg",i,":",2*temp.iloc[i].tavg - temp.iloc[i].tmax)

Avg 0 : 1.0
Avg 1 : 0.0
Avg 2 : 0.0
Avg 3 : 0.0
Avg 4 : 1.0


Averages for the above were wrongly calculated based on an assumed ZERO "tmin" for mising values.

In [22]:
weather.drop("tavg",axis = 1, inplace= True)

In [23]:
weather[(weather.tmin.isnull()) & (weather.station_nbr == 20)]

,station_nbr,tmax,tmin,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
date,,,,,,,,,,,,,,,,,,
2012-05-31,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,29.13,NaN,NaN,NaN,NaN
2013-12-06,20,33.0,NaN,NaN,NaN,NaN,48.0,0.0,NaN,NaN,FZRA BR,NaN,0.68,NaN,NaN,NaN,NaN,14.8
2013-12-07,20,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BR HZ,0.0,0.00,NaN,NaN,NaN,NaN,8.5
2013-12-30,20,NaN,NaN,NaN,19.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.3,6.0,32.0,NaN


In [24]:
weather.loc[weather.station_nbr == 20].loc["2012-05-30":"2012-06-01"]

,station_nbr,tmax,tmin,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
date,,,,,,,,,,,,,,,,,,
2012-05-30,20,91.0,68.0,NaN,63.0,69.0,0.0,15.0,NaN,NaN,TSRA RA,0.0,0.35,29.10,29.81,5.8,12.0,11.1
2012-05-31,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,29.13,NaN,NaN,NaN,NaN
2012-06-01,20,87.0,58.0,NaN,50.0,61.0,0.0,8.0,NaN,NaN,NaN,0.0,0.00,29.23,29.94,4.3,8.0,6.3


In [25]:
st20 = weather[(weather.tmin.isnull()) & (weather.station_nbr == 20)].index
st20

DatetimeIndex(['2012-05-31', '2013-12-06', '2013-12-07', '2013-12-30'], dtype='datetime64[ns]', name='date', freq=None)

In [26]:
stations = weather.station_nbr.value_counts().index
stations

Int64Index([15, 14, 1, 17, 2, 18, 3, 19, 4, 20, 16, 6, 7, 8, 9, 10, 11, 12, 13,
            5],
           dtype='int64')

In [27]:
for i in range(stations.size):
    weather.loc[weather.station_nbr == stations[i]] = weather.loc[weather.station_nbr == stations[i]]\
    .interpolate(method='time',limit_direction = "both")

In [28]:
weather.loc[weather.station_nbr == 20].loc[st20]

,station_nbr,tmax,tmin,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
date,,,,,,,,,,,,,,,,,,
2012-05-31,20,89.0,63.000000,NaN,56.5,65.0,0.0,11.5,NaN,NaN,NaN,0.0,0.00,29.1300,29.875,5.05,10.000000,8.70
2013-12-06,20,33.0,28.666667,NaN,24.0,30.6,48.0,0.0,NaN,NaN,FZRA BR,0.0,0.68,29.4125,30.134,10.80,32.333333,14.80
2013-12-07,20,27.0,27.333333,NaN,23.0,29.2,41.5,0.0,NaN,NaN,BR HZ,0.0,0.00,29.4650,30.208,6.10,29.666667,8.50
2013-12-30,20,52.5,29.000000,NaN,19.0,30.0,24.0,0.0,NaN,NaN,NaN,0.0,0.00,29.5000,30.300,6.00,32.000000,6.95


In [29]:
weather.loc[weather.station_nbr == 20].loc["2012-05-30":"2012-06-01"]

,station_nbr,tmax,tmin,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
date,,,,,,,,,,,,,,,,,,
2012-05-30,20,91.0,68.0,NaN,63.0,69.0,0.0,15.0,NaN,NaN,TSRA RA,0.0,0.35,29.10,29.810,5.80,12.0,11.1
2012-05-31,20,89.0,63.0,NaN,56.5,65.0,0.0,11.5,NaN,NaN,NaN,0.0,0.00,29.13,29.875,5.05,10.0,8.7
2012-06-01,20,87.0,58.0,NaN,50.0,61.0,0.0,8.0,NaN,NaN,NaN,0.0,0.00,29.23,29.940,4.30,8.0,6.3


In [31]:
new_null = weather.isnull().sum().sum()
new_null

44444

In [32]:
org_null - new_null

6664

In [33]:
weather.shape

(19695, 18)

In [34]:
weather[weather.wetbulb.isnull()].station_nbr.value_counts()

5    30
Name: station_nbr, dtype: int64

In [35]:
weather[weather.stnpressure.isnull()].station_nbr.value_counts()

5    30
Name: station_nbr, dtype: int64

Station 5 did not record "wetbulb" and "stpressure" values.

In [36]:
weather[weather.depart.isnull()].loc[:].station_nbr.value_counts()

7     1035
13    1035
20    1035
12    1035
10    1035
17    1035
9     1035
1     1035
16    1035
8     1035
Name: station_nbr, dtype: int64

In [37]:
weather[weather.depart.isnull() == False].loc[:].station_nbr.value_counts()

15    1035
14    1035
6     1035
4     1035
19    1035
11    1035
3     1035
18    1035
2     1035
5       30
Name: station_nbr, dtype: int64

Half of the stations are not recordig depart values.

In [38]:
weather.to_csv("WeatherCleanNoEncoding.csv")

In [39]:
weather = pd.read_csv("WeatherCleanNoEncoding.csv",index_col="date",parse_dates=True)
weather.head()

,station_nbr,tmax,tmin,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
date,,,,,,,,,,,,,,,,,,
2012-01-01,1,52.0,31.0,NaN,36.0,40.0,23.0,0.0,NaN,NaN,RA FZFG BR,NaN,0.05,29.78,29.92,3.6,20.0,4.6
2012-01-01,2,48.0,33.0,16.0,37.0,39.0,24.0,0.0,716.0,1626.0,RA,0.0,0.07,28.82,29.91,9.1,23.0,11.3
2012-01-01,3,55.0,34.0,9.0,24.0,36.0,20.0,0.0,735.0,1720.0,NaN,0.0,0.00,29.77,30.47,9.9,31.0,10.0
2012-01-01,4,63.0,47.0,4.0,28.0,43.0,10.0,0.0,728.0,1742.0,NaN,0.0,0.00,29.79,30.48,8.0,35.0,8.2
2012-01-01,6,63.0,34.0,0.0,31.0,43.0,16.0,0.0,727.0,1742.0,NaN,0.0,0.00,29.95,30.47,14.0,36.0,13.8
